In [11]:
from CryptoPlus.Cipher import python_Serpent

# dicts for testing
from CryptoPlus.testvectors import dict_serpent128,dict_serpent192,dict_serpent256

from tqdm import tqdm
import os
import sys
import re
import numpy as np

In [2]:
def cipher(plaintext, key, IV=None, segment_size=16):
    cipher = python_Serpent.new(key, python_Serpent.MODE_CFB, IV, segment_size=segment_size)
    ciphertext = cipher.encrypt(plaintext)
    return ciphertext


def decipher(ciphertext, key, IV=None):
    decipher = python_Serpent.new(key, python_Serpent.MODE_CFB, IV, segment_size=16)
    deciphered = decipher.decrypt(ciphertext)
    return deciphered

## test

In [3]:
to_test = False

if to_test:
    # for d in dict_serpent128, dict_serpent192, dict_serpent256
    d = dict_serpent128
    print(len(d), len(d) / 3)
    for i in tqdm(range(len(d) // 3)):
        msg = d['msg%i' % i]
        key = d['key%i' % i]
        cip = d['cip%i' % i]
        t = (msg, key, cip)
        t_h = list(map(lambda x: bytes.fromhex(x), t))
        ciphertext = cipher(t_h[0], t_h[1], IV, segment_size=16)
        deciphered = decipher(ciphertext, t_h[1])
        if not (deciphered == t_h[0]):
            print(i)
            print(t)
            print(t_h)
            print(ciphertext, deciphered)
            print()
    print("end")

In [4]:
# iv = bytes.fromhex('00' * 16)
# print(iv, len(iv))

In [5]:
i = 510

msg = dict_serpent256['msg%i' % i]
key = dict_serpent256['key%i' % i]
cip = dict_serpent256['cip%i' % i]
t = (msg, key, cip)
t_h = list(map(lambda x: bytes.fromhex(x), t))
print(t)
print(t_h)

ciphertext = cipher(t_h[0], t_h[1], segment_size=16)
deciphered = decipher(ciphertext, t_h[1])
print(ciphertext)
print(t_h[2])
print(deciphered.hex().upper(), deciphered)
print(deciphered == t_h[0])

('7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E', '7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E', 'C77C11CBD96C18F7633552AE6380C370')
[b'~~~~~~~~~~~~~~~~', b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~', b'\xc7|\x11\xcb\xd9l\x18\xf7c5R\xaec\x80\xc3p']
b'\x84\xfb\xfbNb\xd3t\xff\x17\xb7\xfe\x1dd:\x01\x9e'
b'\xc7|\x11\xcb\xd9l\x18\xf7c5R\xaec\x80\xc3p'
7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E7E b'~~~~~~~~~~~~~~~~'
True


## read books

In [6]:
original_texts_dp = '../books'
original_texts = {}
for fn in os.listdir(original_texts_dp):
    name = fn.split('.')[0]
    with open(os.path.join(original_texts_dp, fn)) as file:
        text = file.read()
        text = re.sub(r'[^\x00-\x7f]', '', text) # remove non-ascii symbols
        original_texts[name] = text
        print('read "%s". len: %d. size: %.3f kBytes' % (fn, len(text), sys.getsizeof(text) / 1024))

read "dracula.txt". len: 867096. size: 846.821 kBytes
read "modest_proposal.txt". len: 38965. size: 38.100 kBytes
read "tom_sawyer.txt". len: 409623. size: 400.070 kBytes
read "dr_jekyll.txt". len: 137671. size: 134.492 kBytes
read "sherlock_holmes.txt". len: 575061. size: 561.631 kBytes
read "pride_and_prejudice.txt". len: 700637. size: 684.264 kBytes
read "moby_dick.txt". len: 1230408. size: 1201.618 kBytes


In [7]:
original_texts.keys()

dict_keys(['dracula', 'modest_proposal', 'tom_sawyer', 'dr_jekyll', 'sherlock_holmes', 'pride_and_prejudice', 'moby_dick'])

## manual example

In [8]:
sample = original_texts['modest_proposal'][1000:1500]
print(sample)

h this great town,
or travel in the country, when they see the streets, the roads and
cabbin-doors crowded with beggars of the female sex, followed by three,
four, or six children, all in rags, and importuning every passenger for
an alms. These mothers instead of being able to work for their honest
livelihood, are forced to employ all their time in stroling to beg
sustenance for their helpless infants who, as they grow up, either turn
thieves for want of work, or leave their dear native country,


In [9]:
sample_bytes = sample.encode('utf-8')
sample_bytes

b'h this great town,\nor travel in the country, when they see the streets, the roads and\ncabbin-doors crowded with beggars of the female sex, followed by three,\nfour, or six children, all in rags, and importuning every passenger for\nan alms. These mothers instead of being able to work for their honest\nlivelihood, are forced to employ all their time in stroling to beg\nsustenance for their helpless infants who, as they grow up, either turn\nthieves for want of work, or leave their dear native country,'

In [18]:
def generate_random_key(bits_cnt):
    nums = np.random.randint(256, size=(bits_cnt // 16))
    key = ''.join(['{:02x}'.format(x).upper() for x in nums])
    key = key.encode('utf-8')
    return key

In [57]:
key = generate_random_key(128)
print(key)
print(len(key))

b'4F357BA9F60314A5'
16


In [58]:
print('key: %r\nkey hex: %r\nkey len: %d\n' % (key, key.hex().upper(), len(key)))

plaintext = sample_bytes
print('plaintext len: ', len(plaintext))
ciphertext = cipher(plaintext, key)
print('ciphertext len: ', len(ciphertext))
print('\nciphertext\n%r' % ciphertext)

key: b'4F357BA9F60314A5'
key hex: '34463335374241394636303331344135'
key len: 16

plaintext len:  500
ciphertext len:  500

ciphertext
b';\xac\xfc\xaf\xf7r\x92c~U\x18\\7\x81fI6\x00a\tD,\xa8\xb6\x11\xc0S5\x87e`\xee%8\x96Y\xb2\x93\x00\xf6y\x99\xc9\xc3\x92<\xf7\xe2\x08g\xed*\x9f\xef\xf6X\xb93Sa,\x9f\xbdK3\x06E\xa3\x89\xc4\xaf\xea4Z\xc6\xa4\xcc\xc9FP3\x9e\x7f\xcf\xf8\x01q\x11\x85w\x8f\xb8\xb9\x0c\xb0\xdc\t\xeb%x\xaf\xd5\x1f,\xf4"c\x1a\x86{\xa7O\x0c\x1a\x9cdnQ\xc4\xf37\x14\xcd\xda5>|\x83\xabV3\x98\xb4\xaf\x03n\x0c\xa2\xb3\xb90\xf7\xab\\\x87g\xba\xb7\xcbLJ\xfa\xc3\xefZ\xc0\xce5\xf9`\xb7\xbb\x9a\xaf\xc2\x10e\x8e\xa8\xc3\xc0\xea<\x19\xc2\xbaL`=\xb6\x1f\xd6\xc7>\xf4x\xde%\xde\xc0\x84\x85\xdc\xdf\x84(\x1bBn\xed\x86\x1c\xe8"\xf1\x16)3*\xf72\xa6*\xdf\x86&w?L\x00\xca\x07\xd8\xac\xf0&\xdc!s\xe4\x07\x9dN\xa3\x8aGeO\xa5\x8e?Y-Q]\x05]\xe3Oz\xeed\xb6h\xe27\xa9\x1d\x8bm\xabT\xf3\x02P\xe6ju\xdf\xe6Tc\xf0\xa5\xe5\x8d\xe2"\x15k\xeb\x08\x19\x8d\xdfa,\x97Mn_KuQ\x9a\x91Zm\xc1a\x17\x9b\x90\x0cy`\xb4h\x15\xa7\x1

In [59]:
deciphered = decipher(ciphertext, key)
success = (plaintext == deciphered)
print('success: %r' % success)

success: True


In [60]:
print(deciphered.decode())

h this great town,
or travel in the country, when they see the streets, the roads and
cabbin-doors crowded with beggars of the female sex, followed by three,
four, or six children, all in rags, and importuning every passenger for
an alms. These mothers instead of being able to work for their honest
livelihood, are forced to employ all their time in stroling to beg
sustenance for their helpless infants who, as they grow up, either turn
thieves for want of work, or leave their dear native country,
